In [2]:
import pandas as pd

In [4]:
from sqlalchemy import create_engine

username = 'ubuntu'

params = { 'host': '18.216.176.43',
           'port': 5432
}

connection_string = f'postgres://{username}:{params["host"]}@{params["host"]}:{params["port"]}/faa'

In [5]:
engine = create_engine(connection_string)

In [8]:
db = pd.read_sql_query('select * from ord_w_wthr', engine)

In [9]:
db.shape

(1090649, 127)

In [10]:
cols = db.columns.tolist()

In [22]:
wthr_cols = cols[-14:-8]

In [28]:
cols[0:15]

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'DOT_ID_Reporting_Airline',
 'IATA_CODE_Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin']

In [64]:
features = cols[1:3] + cols[4:7] + cols[10:11] + cols[14:15] + cols[23:24] \
+ cols[27:28] + cols[30:34] + cols[54:55]  + cols[113:119]

features

['Quarter',
 'Month',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'Flight_Number_Reporting_Airline',
 'Origin',
 'Dest',
 'DestStateName',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'Distance',
 'awnd',
 'pgtm',
 'prcp',
 'snow',
 'snwd',
 'tavg']

In [65]:
db[features].head()

,Quarter,Month,DayOfWeek,FlightDate,Reporting_Airline,Flight_Number_Reporting_Airline,Origin,Dest,DestStateName,DepTime,DepDelay,DepDelayMinutes,DepDel15,Distance,awnd,pgtm,prcp,snow,snwd,tavg
0,1,1,1,2017-01-09,AA,1110,ORD,PHX,Arizona,500.0,0.0,0.0,0.0,1440.0,12.97,None,0.02,0.2,0.0,22
1,1,1,2,2017-01-10,AA,1110,ORD,PHX,Arizona,459.0,-1.0,0.0,0.0,1440.0,23.26,None,0.20,0.0,0.0,38
2,1,1,3,2017-01-11,AA,1110,ORD,PHX,Arizona,451.0,-9.0,0.0,0.0,1440.0,12.08,None,0.33,0.0,0.0,32
3,1,1,4,2017-01-12,AA,1110,ORD,PHX,Arizona,451.0,-9.0,0.0,0.0,1440.0,11.18,None,0.05,0.2,0.0,26
4,1,1,5,2017-01-13,AA,1110,ORD,PHX,Arizona,456.0,-4.0,0.0,0.0,1440.0,8.95,None,0.00,0.0,0.0,20


In [66]:
mini_db = db[features]

In [75]:
pd.DataFrame().tail()

,DepTime
1090644,False
1090645,True
1090646,True
1090647,True
1090648,True


In [76]:
mini_db['MorningFlight'] = (mini_db.DepTime < 1200)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [80]:
#pd.get_dummies(mini_db.DepDelay)
mini_db['Delayed'] = (mini_db.DepDelay > 0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [85]:
mini_db = mini_db.drop(columns=['DepDel15', 'DepDelayMinutes'], axis=1)

In [86]:
mini_db.columns

Index(['Quarter', 'Month', 'DayOfWeek', 'FlightDate', 'Reporting_Airline',
       'Flight_Number_Reporting_Airline', 'Origin', 'Dest', 'DestStateName',
       'DepTime', 'DepDelay', 'Distance', 'awnd', 'pgtm', 'prcp', 'snow',
       'snwd', 'tavg', 'MorningFlight', 'Delayed'],
      dtype='object')

In [91]:
(mini_db['DepDelay'] > 0)

0          False
1          False
2          False
3          False
4          False
5          False
6          False
7          False
8          False
9          False
10         False
11         False
12         False
13         False
14         False
15         False
16         False
17         False
18         False
19         False
20         False
21         False
22          True
23         False
24         False
25         False
26         False
27          True
28         False
29         False
           ...  
1090619     True
1090620     True
1090621     True
1090622     True
1090623    False
1090624     True
1090625    False
1090626     True
1090627     True
1090628    False
1090629    False
1090630    False
1090631    False
1090632     True
1090633    False
1090634    False
1090635     True
1090636    False
1090637     True
1090638     True
1090639    False
1090640    False
1090641     True
1090642    False
1090643     True
1090644     True
1090645     True
1090646     Tr

In [92]:
mini_db['was_delayed'] = (mini_db['DepDelay'] > 0)

In [97]:
mini_db = mini_db.drop(columns=['Delayed'], axis=1)

In [102]:
pd.get_dummies(mini_db.was_delayed)

,False,True
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
5,1,0
6,1,0
7,1,0
8,1,0
9,1,0


In [113]:
mini_db = mini_db.drop(columns=['DepDelay'], axis=1)

In [120]:
mini_db = mini_db.drop(columns=['pgtm'], axis=1)

In [142]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [126]:
mini_db = mini_db.rename(columns={False: 'y'})

In [123]:
mini_db = mini_db.dropna()

In [127]:
mini_db.head()

,y,Quarter,Month,DayOfWeek,FlightDate,Reporting_Airline,Flight_Number_Reporting_Airline,Origin,Dest,DestStateName,DepTime,Distance,awnd,prcp,snow,snwd,tavg,MorningFlight
0,1,1,1,1,2017-01-09,AA,1110,ORD,PHX,Arizona,500.0,1440.0,12.97,0.02,0.2,0.0,22,True
1,1,1,1,2,2017-01-10,AA,1110,ORD,PHX,Arizona,459.0,1440.0,23.26,0.20,0.0,0.0,38,True
2,1,1,1,3,2017-01-11,AA,1110,ORD,PHX,Arizona,451.0,1440.0,12.08,0.33,0.0,0.0,32,True
3,1,1,1,4,2017-01-12,AA,1110,ORD,PHX,Arizona,451.0,1440.0,11.18,0.05,0.2,0.0,26,True
4,1,1,1,5,2017-01-13,AA,1110,ORD,PHX,Arizona,456.0,1440.0,8.95,0.00,0.0,0.0,20,True


In [130]:
mini_db = pd.concat([mini_db, pd.get_dummies(mini_db.MorningFlight)], axis=1)

In [132]:
mini_db = mini_db.rename(columns={True: 'MngFlt_true', False: 'MngFlt_flse'})

In [135]:
mini_db = pd.concat([mini_db, pd.get_dummies(mini_db.Reporting_Airline)], axis=1)

In [136]:
mini_db = mini_db.drop(columns=['DepTime', 'Flight_Number_Reporting_Airline'], axis = 1)

In [137]:
mini_db.shape

(1071720, 35)

In [138]:
mini_db.groupby('y').count()

,Quarter,Month,DayOfWeek,FlightDate,Reporting_Airline,Origin,Dest,DestStateName,Distance,awnd,...,G4,MQ,NK,OH,OO,UA,VX,WN,YV,YX
y,,,,,,,,,,,,,,,,,,,,,
0,341093,341093,341093,341093,341093,341093,341093,341093,341093,341093,...,341093,341093,341093,341093,341093,341093,341093,341093,341093,341093
1,730627,730627,730627,730627,730627,730627,730627,730627,730627,730627,...,730627,730627,730627,730627,730627,730627,730627,730627,730627,730627


In [144]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

In [208]:
y = mini_db['y']
X = mini_db.iloc[:,1:]

Quarter                int64
Month                  int64
DayOfWeek              int64
FlightDate            object
Reporting_Airline     object
Origin                object
Dest                  object
DestStateName         object
Distance             float64
awnd                 float64
prcp                 float64
snow                 float64
snwd                 float64
tavg                   int64
MorningFlight           bool
MngFlt_flse            uint8
MngFlt_true            uint8
9E                     uint8
AA                     uint8
AS                     uint8
B6                     uint8
DL                     uint8
EV                     uint8
F9                     uint8
G4                     uint8
MQ                     uint8
NK                     uint8
OH                     uint8
OO                     uint8
UA                     uint8
VX                     uint8
WN                     uint8
YV                     uint8
YX                     uint8
dtype: object

In [209]:
X = X.drop(columns=['FlightDate',
                    'Reporting_Airline' 
                    ,'Origin'
                    ,'Dest'
                    ,'DestStateName',
                    'MorningFlight'
                    ], axis=1)

In [210]:
y.shape

(1071720,)

In [211]:
y = np.array(y).reshape(-1,1)

In [212]:
X_tr, X_tst, y_tr, y_tst = train_test_split(X, y, random_state=51)

over_smpl = RandomOverSampler(random_state=52)
X_O_smpld, y_O_smpld = over_smpl.fit_resample(X_tr, y_tr)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [169]:
X_O_smpld[0]

array([1, 3, 1, 1440.0, 10.51, 0.24, 3.9, 2.0, 29, False, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=object)

In [195]:
sclr = StandardScaler()
sclr.fit(X_O_smpld)

X_tr_scld = sclr.transform(X_O_smpld)
X_tst_scld = sclr.transform(X_tst)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  """


In [165]:
X.shape

(1071720, 33)

In [104]:
logr = LogisticRegression(C=1)

logr.fit(X_tr_scld, y_O_smpld)

logr.score(X_tst_scld, y_tst)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6083369761368349

In [172]:
logr = LogisticRegression(C=1)

logr.fit(X_tr_scld, y_O_smpld)

logr.score(X_tst_scld, y_tst)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5826745791811294

In [175]:
roc_auc_score(y_tst, logr.predict(X_tst_scld))

0.6085556931268958

In [115]:

#roc_auc_score(y_tst, logr.predict(y_tst))
type(X_tst_scld)

numpy.ndarray

In [116]:
X_tst_scld.shape

(267651, 6)

In [117]:
y_tst.shape

(267651,)

In [118]:
type(y_tst)

pandas.core.series.Series

In [119]:
type(y_O_smpld)

numpy.ndarray

In [121]:
y_O_smpld.shape

(1305702,)

In [127]:
logr.predict()

ValueError: Expected 2D array, got 1D array instead:
array=[0 0 0 ... 1 0 0].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [129]:
roc_auc_score(y_tst, logr.predict(X_tst_scld))

0.6146466223672911

In [196]:
rf = RandomForestClassifier()

rf.fit(X_tr, y_tr)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [197]:
roc_auc_score(y_tst, rf.predict(X_tst))

0.6189090948671396

In [198]:
for i in zip(X.columns, rf.feature_importances_):
    print(i)

('Quarter', 0.0067447812598469435)
('Month', 0.027794585197613457)
('DayOfWeek', 0.037307556206266426)
('Distance', 0.6013896024955407)
('awnd', 0.07423526819607107)
('prcp', 0.05376526285827839)
('snow', 0.008472966998854485)
('snwd', 0.005022648158090914)
('tavg', 0.07627959828600493)
('MngFlt_flse', 0.035595716686183054)
('MngFlt_true', 0.017071556870594272)
('9E', 0.001176037383406972)
('AA', 0.00868892661747667)
('AS', 0.002053318904572709)
('B6', 0.0026495116159222488)
('DL', 0.00427859903782094)
('EV', 0.0037575983676825394)
('F9', 0.003149366692877861)
('G4', 0.0010837253832380073)
('MQ', 0.003808412312927778)
('NK', 0.004744236193011729)
('OH', 0.0016397919574918413)
('OO', 0.007015886780522018)
('UA', 0.007859995875519294)
('VX', 0.0011517438175305359)
('YV', 0.0)
('YX', 0.0032633058466542304)


In [199]:
X.groupby(X.AA).count()

,Quarter,Month,DayOfWeek,Distance,awnd,prcp,snow,snwd,tavg,MngFlt_flse,...,F9,G4,MQ,NK,OH,OO,UA,VX,YV,YX
AA,,,,,,,,,,,,,,,,,,,,,
0,840625,840625,840625,840625,840625,840625,840625,840625,840625,840625,...,840625,840625,840625,840625,840625,840625,840625,840625,840625,840625
1,231095,231095,231095,231095,231095,231095,231095,231095,231095,231095,...,231095,231095,231095,231095,231095,231095,231095,231095,231095,231095


In [200]:
231095/840625

0.2749085501858736

In [201]:
db.groupby(db.Reporting_Airline).count()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,...,snwd,tavg,wt01,wt02,wt03,wt04,wt05,wt06,wt08,wt09
Reporting_Airline,,,,,,,,,,,,,,,,,,,,,
9E,3337,3337,3337,3337,3337,3337,3337,3337,3337,3337,...,3337,3337,1508,146,477,66,0,40,995,18
AA,234402,234402,234402,234402,234402,234402,234402,234402,234314,234402,...,234402,234402,96783,8498,36544,3027,1158,1670,70831,1351
AS,9407,9407,9407,9407,9407,9407,9407,9407,9407,9407,...,9407,9407,3934,307,1615,104,39,56,2954,40
B6,9157,9157,9157,9157,9157,9157,9157,9157,9157,9157,...,9157,9157,3739,313,1563,100,54,58,2800,42
DL,31154,31154,31154,31154,31154,31154,31154,31154,31154,31154,...,31154,31154,12853,1072,5073,362,161,210,9868,126
EV,62546,62546,62546,62546,62546,62546,62546,62546,62546,62546,...,62546,62546,25547,2644,9959,921,427,575,16829,344
F9,10787,10787,10787,10787,10787,10787,10787,10787,10669,10787,...,10787,10787,4523,411,1702,143,54,81,3196,60
G4,1647,1647,1647,1647,1647,1647,1647,1647,1643,1647,...,1647,1647,742,58,248,20,0,10,508,10
MQ,99012,99012,99012,99012,99012,99012,99012,99012,99012,99012,...,99012,99012,45720,3837,15244,1468,0,923,31483,632


In [204]:
234402 / 1090649

0.2149197404481185

In [203]:
db.describe()

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,...,prcp,snow,snwd,tavg,wt01,wt02,wt03,wt04,wt06,wt08
count,1.090649e+06,1.090649e+06,1.090649e+06,1.090649e+06,1.090649e+06,1.090649e+06,1.090649e+06,1.090649e+06,1.090649e+06,1.090649e+06,...,1.090649e+06,1.090649e+06,1.090649e+06,1.090649e+06,454492.0,38918.0,173112.0,14120.0,7862.0,330388.0
mean,2.017511e+03,2.396610e+00,6.141048e+00,1.575958e+01,3.924864e+00,2.012158e+04,2.506774e+03,1.329329e+04,1.329333e+06,3.146272e+04,...,1.316386e-01,6.454643e-02,1.983819e-01,5.497371e+01,1.0,1.0,1.0,1.0,1.0,1.0
std,4.998717e-01,1.037816e+00,3.112286e+00,8.760093e+00,1.993751e+00,2.464191e+02,1.699780e+03,1.260675e+03,1.260679e+05,1.083379e+03,...,3.548006e-01,4.149319e-01,1.098433e+00,1.963133e+01,0.0,0.0,0.0,0.0,0.0,0.0
min,2.017000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.939300e+04,2.000000e+00,1.013500e+04,1.013503e+06,3.013500e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0
25%,2.017000e+03,2.000000e+00,4.000000e+00,8.000000e+00,2.000000e+00,1.993000e+04,1.023000e+03,1.288900e+04,1.288903e+06,3.097700e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,3.900000e+01,1.0,1.0,1.0,1.0,1.0,1.0
50%,2.018000e+03,2.000000e+00,6.000000e+00,1.600000e+01,4.000000e+00,1.997700e+04,2.311000e+03,1.393000e+04,1.393004e+06,3.097700e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,5.900000e+01,1.0,1.0,1.0,1.0,1.0,1.0
75%,2.018000e+03,3.000000e+00,9.000000e+00,2.300000e+01,6.000000e+00,2.030400e+04,3.715000e+03,1.393000e+04,1.393006e+06,3.170300e+04,...,7.000000e-02,0.000000e+00,0.000000e+00,7.200000e+01,1.0,1.0,1.0,1.0,1.0,1.0
max,2.018000e+03,4.000000e+00,1.200000e+01,3.100000e+01,7.000000e+00,2.117100e+04,7.362000e+03,1.591900e+04,1.591904e+06,3.545400e+04,...,4.190000e+00,6.200000e+00,1.180000e+01,8.700000e+01,1.0,1.0,1.0,1.0,1.0,1.0


In [217]:
from sklearn.naive_bayes import GaussianNB

In [218]:
gnb = GaussianNB()

In [214]:
gnb.fit(X, y)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Input X must be non-negative